## 1. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [22]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from torch.optim.lr_scheduler import CosineAnnealingLR
import wandb
import datetime

In [23]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [24]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, df, path, transform=None):
        self.df = df.values if isinstance(df, pd.DataFrame) else pd.read_csv(df).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [25]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device, epoch=None):
    model.train()
    train_loss = 0
    preds_list, targets_list = [], []

    pbar = tqdm(loader, desc=f"Train Epoch {epoch+1}" if epoch is not None else "Train")
    for image, targets in pbar:
        image, targets = image.to(device), targets.to(device)

        optimizer.zero_grad(set_to_none=True)
        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).cpu().numpy())
        targets_list.extend(targets.cpu().numpy())

    # ---- epoch별 평균 계산 ----
    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    # ---- wandb 로그 기록 ----
    wandb.log({
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
        "lr": optimizer.param_groups[0]["lr"],        # ✅ 학습률 로그 추가
        "epoch": epoch + 1 if epoch is not None else 0
    })

    return {"train_loss": train_loss, "train_acc": train_acc, "train_f1": train_f1}


# Validation용 함수 추가
def valid_one_epoch(loader, model, loss_fn, device, epoch=None):
    model.eval()
    val_loss = 0
    preds_list, targets_list = [], []

    with torch.no_grad():
        pbar = tqdm(loader, desc=f"Valid Epoch {epoch+1}" if epoch is not None else "Valid")
        for image, targets in pbar:
            image, targets = image.to(device), targets.to(device)
            preds = model(image)
            loss = loss_fn(preds, targets)
            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).cpu().numpy())
            targets_list.extend(targets.cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    # ---- wandb 로그 기록 ----
    wandb.log({
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
        "epoch": epoch + 1 if epoch is not None else 0
    })

    return {"val_loss": val_loss, "val_acc": val_acc, "val_f1": val_f1}


## 2. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [26]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = 'datasets_fin/'

# model config
model_name = 'resnet34' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 640 # 224
LR = 1e-3
EPOCHS = 10
BATCH_SIZE = 32
num_workers = 0

## 3. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [27]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [29]:
print("총 이미지 수:", len(os.listdir("../data/train_balanced")))
df = pd.read_csv("../data/train_balanced.csv")
print(df["target"].value_counts().sort_index())


총 이미지 수: 1700
target
0     100
1     100
2     100
3     100
4     100
5     100
6     100
7     100
8     100
9     100
10    100
11    100
12    100
13    100
14    100
15    100
16    100
Name: count, dtype: int64


In [30]:
# --- train/validation split 추가 ---
# train_df = pd.read_csv("../data/train.csv")
train_df = pd.read_csv("../data/train_balanced.csv")
train_df, val_df = train_test_split(
    train_df,
    test_size=0.2,          # ✅ 80:20 비율로 train/val split
    stratify=train_df['target'],  # ✅ 클래스 비율 유지
    random_state=42
)

# Dataset 정의
# trn_dataset = ImageDataset(train_df, "../data/train/", transform=trn_transform)
# val_dataset = ImageDataset(val_df, "../data/train/", transform=tst_transform)
trn_dataset = ImageDataset(train_df, "../data/train_balanced/", transform=trn_transform)
val_dataset = ImageDataset(val_df, "../data/train_balanced/", transform=tst_transform)

tst_dataset = ImageDataset(
    "../data/sample_submission.csv",
    "../data//test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

1360 3140


In [31]:
# DataLoader 정의
trn_loader = DataLoader(trn_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)  # ✅ 추가
tst_loader = DataLoader(tst_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

## 4. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [31]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

# ✅ CosineAnnealingLR 스케줄러 추가
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

In [32]:
run_name = f"{model_name}_{datetime.datetime.now().strftime('%m%d_%H%M')}"
wandb.init(project="document-type-classification", name=run_name)  # ✅ wandb 초기화

for epoch in range(EPOCHS):
    train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device, epoch=epoch)
    val_metrics = valid_one_epoch(val_loader, model, loss_fn, device=device, epoch=epoch)

    # ✅ CosineAnnealingLR 스케줄러 업데이트
    scheduler.step()
    
    print(
        f"[Epoch {epoch+1}/{EPOCHS}] "
        f"Train Loss: {train_metrics['train_loss']:.4f}, "
        f"Train F1: {train_metrics['train_f1']:.4f}, "
        f"Val Loss: {val_metrics['val_loss']:.4f}, "
        f"Val F1: {val_metrics['val_f1']:.4f}, "
        f"LR: {optimizer.param_groups[0]['lr']:.8f}"
    )

wandb.finish() # 모든 로그 전송 후 clean-up

Valid Epoch 1: 100%|██████████| 10/10 [00:01<00:00,  9.67it/s]


[Epoch 1/10] Train Loss: 2.5651, Train F1: 0.1874, Val Loss: 2.0430, Val F1: 0.3470, LR: 0.00097555


Valid Epoch 2: 100%|██████████| 10/10 [00:01<00:00,  9.50it/s]


[Epoch 2/10] Train Loss: 1.8480, Train F1: 0.3947, Val Loss: 1.6514, Val F1: 0.4367, LR: 0.00090460


Valid Epoch 3: 100%|██████████| 10/10 [00:01<00:00,  9.59it/s]


...

Valid Epoch 9: 100%|██████████| 10/10 [00:01<00:00,  8.50it/s]


[Epoch 9/10] Train Loss: 0.6533, Train F1: 0.7753, Val Loss: 0.8698, Val F1: 0.6748, LR: 0.00002545


Valid Epoch 10: 100%|██████████| 10/10 [00:01<00:00,  9.61it/s]

[Epoch 10/10] Train Loss: 0.6384, Train F1: 0.7715, Val Loss: 0.8825, Val F1: 0.6506, LR: 0.00000100


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
lr,██▇▇▆▄▃▂▂▁
train_acc,▁▄▅▆▆▇▇███
train_f1,▁▃▄▅▆▇▇███
train_loss,█▅▄▃▂▂▂▁▁▁
val_acc,▁▃▅▆▆████▇
val_f1,▁▃▅▆▆████▇
val_loss,█▆▄▃▂▂▁▁▁▁
epoch,10
lr,3e-05
train_acc,0.79777


# 5. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [33]:
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:10<00:00,  9.13it/s]


In [34]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [35]:
sample_submission_df = pd.read_csv("../data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [36]:
pred_df.to_csv("pred.csv", index=False)

In [37]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,10
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,6
4,00901f504008d884.jpg,2
